In [1]:
import os
import yaml

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\MLFlow\\End-To-End-DSProject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\HP\\Desktop\\MLFlow\\End-To-End-DSProject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml , create_directories

[2025-06-02 23:52:09,560: INFO: __init__: Logger initialized successfully in datascience package.]


In [6]:
from src.datascience.constants import CONFIG_FILEPATH
print(CONFIG_FILEPATH)


config\config.yaml


In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILEPATH,
                 params_filepath=PARAMS_FILEPATH,
                 schema_filepath=SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [8]:
import os
import requests
from src.datascience import logger
import zipfile

In [9]:
#Component-DataIngestion

class DataIngestion:
    def __init__(self , config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_url, stream=True)
            response.raise_for_status()  # Raises HTTPError if not 200
            with open(self.config.local_data_file, "wb") as f:
                f.write(response.content)
            logger.info(f"{self.config.local_data_file} downloaded successfully!")
        else:
            logger.info("File already exists")


    def extract_zipfile(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e


[2025-06-02 23:52:12,196: INFO: common: yaml file:config\config.yaml is loaded successfully]
[2025-06-02 23:52:12,209: INFO: common: yaml file:params.yaml is loaded successfully]
[2025-06-02 23:52:12,220: INFO: common: yaml file:schema.yaml is loaded successfully]
[2025-06-02 23:52:12,230: INFO: common: Created directory at :artifacts_root]


[2025-06-02 23:52:12,237: INFO: common: Created directory at :artifacts/data_ingestion]
[2025-06-02 23:52:12,755: INFO: 3841572504: artifacts/data_ingestion/data.zip downloaded successfully!]
